In [1]:
import requests

WIKI_API_URL = "https://ja.wikipedia.org/w/api.php"

# 検索キーワードのページ取得
def fetch_search_results(keyword, limit=10):
    params = {
        "action": "query",
        "list": "search",
        "srsearch": keyword,
        "format": "json",
        "srlimit": limit
    }

    headers = {
        "User-Agent": "LYGroup-WikiClient/1.0 (sarimura@lycorp.co.jp)"  # ←識別可能なUA
    }

    try:
        response = requests.get(WIKI_API_URL, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        return data.get("query", {}).get("search", [])
    except requests.RequestException as e:
        print("検索リクエストでエラーが発生しました:", e)
        return []

# ページ内容を取得
def fetch_page_content(title):
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True
    }

    headers = {
        "User-Agent": "LYGroup-WikiClient/1.0 (sarimura@lycorp.co.jp)"
    }
    

    try:
        response = requests.get(WIKI_API_URL, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        page = next(iter(data.get("query", {}).get("pages", {}).values()), {})
        return page.get("extract", "ページの内容が見つかりません")
    except requests.RequestException as e:
        print("ページ内容取得でエラーが発生しました:", e)
        return "ページ内容が取得できませんでした。"

In [2]:
# テスト
keyword = "Python"

search_results = fetch_search_results(keyword)
if search_results:
    print("【キーワードに該当するページ一覧】")
    for result in search_results[:3]:
        print(f"- {result['title']}")
    
    # 最初のページタイトル取得
    first_title = search_results[0]['title']
    print("\n【最初のページタイトル】")
    print(first_title)

    # 最初のページ内容
    page_content = fetch_page_content(first_title)
    print("\n【ページ内容】")
    print(page_content[:500])
else:
    print("検索結果が見つかりません。")

【キーワードに該当するページ一覧】
- Python
- モンティ・パイソン
- IronPython

【最初のページタイトル】
Python

【ページ内容】
Python（パイソン）はインタープリタ型の高水準汎用プログラミング言語である。


== 概要 ==
Pythonは1991年にグイド・ヴァン・ロッサムにより開発されたプログラミング言語である。
最初にリリースされたPythonの設計哲学は、ホワイトスペース（オフサイドルール）の顕著な使用によってコードの可読性を重視している。その言語構成とオブジェクト指向のアプローチは、プログラマが小規模なプロジェクトから大規模なプロジェクトまで、明確で論理的なコードを書くのを支援することを目的としている。
Pythonは動的に型付けされていて、ガベージコレクションされている。構造化（特に手続き型）、オブジェクト指向、関数型プログラミングを含む複数のプログラミングパラダイムをサポートしている。Pythonは、その包括的な標準ライブラリのため、しばしば「バッテリーを含む」言語と表現されている。
Pythonのインタプリタは多くのOSに対応している。プログラマーのグローバルコミュニティは、自由かつオープンソース  のリファレンス実装であるCPythonを開発および保守している 。非営利団体であるPyth


In [9]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv("../.env")

client = OpenAI(api_key=os.environ['API_KEY'])

MODEL_NAME = "gpt-4o-mini"
#MODEL_NAME = "gpt-4o"

In [10]:
# 要約を行うプロンプトを作成
prompt = f"""
以下の文章を要約してください。

# 条件：
- 小学生にもわかるように
- 300文字程度

# 文章：
{page_content[:1000]}
"""

# APIリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
)

In [13]:
# 回答表示
print(response.choices[0].message.content.strip())

LINEヤフー株式会社は、日本の大きなインターネット会社で、東京都に本社があります。主に「Yahoo! JAPAN」というウェブサイトや「LINE」というメッセージアプリを運営しています。この会社は、ソフトバンクと韓国のネイバー社が一緒に作った会社です。

1996年に、アメリカのYahooと日本のソフトバンクが協力してヤフー株式会社を設立しました。その後、いくつかの変化があり、2019年には持株会社に移行しました。2021年にはLINEと合併し、「新生Zホールディングス」ができました。そして2023年には、会社名をLINEヤフー株式会社に変更しました。

この会社は、日本の株式市場でも重要な存在で、多くの人々に利用されています。


In [11]:
# 類似ワード調査

import time

keyword = "Python"

search_results = fetch_search_results(keyword)
if search_results:
    print("【キーワードに該当するページ一覧】")
    for result in search_results[:2]:
        print(f"- {result['title']}")
    
        """
        # 検索結果の内、最大2件を表示
        first_title = search_results[0]['title']
        print("\n【最初のページタイトル】")
        print(first_title)
        """

        # 
        page_content = fetch_page_content(result['title'])
        print("【ページ内容】")
        print(page_content[:500])

        # 要約を行うプロンプトを作成
        prompt = f"""
        以下の文章を要約してください。

        # 条件：
        - 読者はマーケティング担当者
        - 箇条書きで3行

        # 文章：
        {page_content[:100]}
        """

        # APIリクエスト
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "user", "content": prompt},
            ],
            max_tokens=100,
            temperature=0.3
        )
        print(response.choices[0].message.content.strip())

        time.sleep(60) 
else:
    print("検索結果が見つかりません。")

【キーワードに該当するページ一覧】
- Python
【ページ内容】
Python（パイソン）はインタープリタ型の高水準汎用プログラミング言語である。


== 概要 ==
Pythonは1991年にグイド・ヴァン・ロッサムにより開発されたプログラミング言語である。
最初にリリースされたPythonの設計哲学は、ホワイトスペース（オフサイドルール）の顕著な使用によってコードの可読性を重視している。その言語構成とオブジェクト指向のアプローチは、プログラマが小規模なプロジェクトから大規模なプロジェクトまで、明確で論理的なコードを書くのを支援することを目的としている。
Pythonは動的に型付けされていて、ガベージコレクションされている。構造化（特に手続き型）、オブジェクト指向、関数型プログラミングを含む複数のプログラミングパラダイムをサポートしている。Pythonは、その包括的な標準ライブラリのため、しばしば「バッテリーを含む」言語と表現されている。
Pythonのインタプリタは多くのOSに対応している。プログラマーのグローバルコミュニティは、自由かつオープンソース  のリファレンス実装であるCPythonを開発および保守している 。非営利団体であるPyth
- Pythonは1991年にグイド・ヴァン・ロッサムによって開発された高水準の汎用プログラミング言語。  
- インタープリタ型であり、コードの実行が容易であるため、学習コストが低い。  
- マーケティング分野でもデータ分析や自動化に活用されている。
- モンティ・パイソン
【ページ内容】
モンティ・パイソン（Monty Python）は、イギリスを代表するコメディグループ。グレアム・チャップマン、ジョン・クリーズ、テリー・ギリアム、エリック・アイドル、テリー・ジョーンズ、マイケル・ペイリンの6人で構成される（ただし、ニール・イネスとキャロル・クリーヴランドを「7人目のパイソン」と表現することもある）。明らかにモンティ・パイソンを話題にしている場合、単にパイソンズと言うこともある。
1969年から始まったBBCテレビ番組『空飛ぶモンティ・パイソン』で人気を博し、その後もライブ、映画、アルバム、書籍、舞台劇等で活躍の場を広げ、その爆発的なインパクトはメンバー個人をスターの座に押し上げた。そのスケッチとスケッチの境